In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from mvn.pipeline.setup import setup_dataloaders, setup_experiment, build_env
from mvn.pipeline.training import do_train

In [3]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [7]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model (shared):
                      backbone has      10896 params (~  0.1) %
                    R_backbone has    5716486 params (~ 76.2) %
                    t_backbone has    1774339 params (~ 23.7) %
total params:              7501721
cam2cam estimation => adding 7501721 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@17.05.2021-18:54:08
#[do_train]: epoch    0 has started!
#[cam2cam]: training batch iter 0 losses: GEO ~ 0.001, TRANS ~ 0.000, POSE ~ 0.002, ROTO ~ 0.000, 3D ~ 0.000, SELF R ~ 0.000, SELF t ~ 0.000, TOTAL ~ 0.007
#[cam2cam]: training batch iter 0 MPJPE: ~ 0.075 mm
#[cam2cam]: cannot backpropagate ... are you cheating?
#[cam2cam]: training batch iter 1 losses: GEO ~ 0.001, TRANS ~ 0.000, POSE ~ 0.003, ROTO ~ 0.000, 3D ~ 0.000, SELF R ~ 0.000, SELF t ~ 0.000, TOTAL ~ 0.006
#[cam2cam]: training batch iter 1 MPJPE: ~ 0.073 mm
#[cam2cam]: cannot backpropagate ... are you che